In [2]:
# importing necassery packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#%matplotlib inline` is a magic command in Jupyter notebooks that allows plots to be displayed directly within the notebook. 

%matplotlib inline
import matplotlib
import warnings
warnings.filterwarnings('ignore')
matplotlib.rcParams["figure.figsize"] = (20,10)

## Data Cleaning
Remove rows with missing values and understanding dataset

Cleaning
Remove rows with missing values and understanding dataset

In [ ]:
df=pd.read_csv("Bengaluru_House_Data.csv")
print(df.head())
print(df.shape)
print(df.describe())
print(df.columns)

In [ ]:
#removing unwanted features
df1=df.drop(['area_type','availability','society','balcony'],axis='columns') # new dataframe after removing unwanted features
df1.head()
print(df1.groupby('location')['location'].count())
print(df1.columns)

In [ ]:
# data cleaning by removing row with NaN (Not A Number)
#df1.isnull().sum() to count the number of missing values in each column
print(df1.isnull().sum())
df1=df1.dropna()
print(df1.isnull().sum())
print(df1.shape)
print(df1.columns)